In [13]:
import torch
import os
import json

In [14]:
folder_path = "./fewshot_models"

combined_data = []

for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        filepath = os.path.join(folder_path, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                combined_data.append(data)
            except json.JSONDecodeError as e:
                print(f"⚠️ Skipping {filename}: invalid JSON ({e})")

combined_json_str = json.dumps(combined_data, ensure_ascii=False, indent=2)

print(combined_json_str)

[
  {
    "source_checkpoint": "./models/lottery_ticket_qqp_sparsity70_seed42.pt",
    "target_task": "sst2",
    "mode": "headonly",
    "seed": 42,
    "train_acc": 0.609375,
    "val_acc": 0.625,
    "examples": 128
  },
  {
    "source_checkpoint": "./models/ensemble_ticket_sst2_qqp_sparsity47_seed42.pt",
    "target_task": "sst2",
    "mode": "full",
    "seed": 42,
    "train_acc": 0.8359375,
    "val_acc": 0.8125,
    "examples": 128
  },
  {
    "source_checkpoint": "./models/lottery_ticket_qqp_sparsity70_seed999.pt",
    "target_task": "sst2",
    "mode": "headonly",
    "seed": 999,
    "train_acc": 0.4921875,
    "val_acc": 0.4375,
    "examples": 128
  },
  {
    "source_checkpoint": "./models/lottery_ticket_qqp_sparsity70_seed123.pt",
    "target_task": "sst2",
    "mode": "headonly",
    "seed": 123,
    "train_acc": 0.546875,
    "val_acc": 0.5625,
    "examples": 128
  },
  {
    "source_checkpoint": "./models/lottery_ticket_qqp_sparsity70_seed42.pt",
    "target_task":

In [16]:
results = []

summary_dir = "./models"

for fname in os.listdir(summary_dir):
    if not fname.endswith("_summary.json"):
        continue

    path = os.path.join(summary_dir, fname)
    with open(path) as f:
        data = json.load(f)

    task = data["task"]
    seed = data["seed"]
    sparsity = data["sparsity"]
    hist = data["history"]

    train_acc_dense = hist["train_acc"][0]
    val_acc_dense = hist["val_acc"][0]
    train_acc_target = hist["train_acc"][-1]
    val_acc_target = hist["val_acc"][-1]

    results.append({
        "task": task,
        "seed": seed,
        "target_sparsity": sparsity,
        "train_acc_dense": train_acc_dense,
        "val_acc_dense": val_acc_dense,
        "train_acc_target": train_acc_target,
        "val_acc_target": val_acc_target,
    })

df = pd.DataFrame(results)

df = df.sort_values(by=["task", "seed", "target_sparsity"]).reset_index(drop=True)

print(df)

out_path = os.path.join(summary_dir, "lottery_ticket_results_table.csv")
df.to_csv(out_path, index=False)


    task  seed  target_sparsity  train_acc_dense  val_acc_dense  \
0    qqp    42              0.0         0.822447       0.861995   
1    qqp    42              0.7         0.822447       0.861995   
2    qqp   123              0.0         0.820962       0.861283   
3    qqp   123              0.7         0.820962       0.861283   
4    qqp   999              0.0         0.820012       0.860392   
5    qqp   999              0.7         0.820012       0.860392   
6   sst2    42              0.0         0.911921       0.923165   
7   sst2    42              0.6         0.911921       0.923165   
8   sst2   123              0.0         0.908477       0.918578   
9   sst2   123              0.6         0.908477       0.918578   
10  sst2   999              0.0         0.913005       0.916284   
11  sst2   999              0.6         0.913005       0.916284   

    train_acc_target  val_acc_target  
0           0.822447        0.861995  
1           0.979513        0.865736  
2          

In [11]:
fewshot_dir = "./fewshot_models"
records = []

for fname in os.listdir(fewshot_dir):
    if not fname.endswith(".json") or not fname.startswith("fewshot_"):
        continue

    path = os.path.join(fewshot_dir, fname)
    with open(path) as f:
        data = json.load(f)

    target_task = data.get("target_task", "unknown")
    mode = data.get("mode", "unknown")
    seed = data.get("seed", None)
    train_acc = data.get("train_acc", None)
    val_acc = data.get("val_acc", None)
    examples = data.get("examples", None)
    source_checkpoint = data.get("source_checkpoint", "")

    source_type = "unknown"
    source_tasks = "unknown"
    sparsity = None

    m = re.search(r"(lottery_ticket|ensemble_ticket)_([a-zA-Z0-9_]+)_sparsity(\d+)_seed(\d+)", source_checkpoint)
    if m:
        source_type = m.group(1).replace("_ticket", "")
        source_tasks = m.group(2).replace("_", "+")  # e.g., sst2_qqp → sst2+qqp
        sparsity = float(m.group(3)) / 100.0

    records.append({
        "target_task": target_task,
        "source_type": source_type,
        "source_tasks": source_tasks,
        "sparsity": sparsity,
        "seed": seed,
        "mode": mode,
        "train_acc": train_acc,
        "val_acc": val_acc,
        "examples": examples,
    })

df = pd.DataFrame(records)
df = df.sort_values(by=["target_task", "source_type", "sparsity", "mode", "seed"]).reset_index(drop=True)

print(df)

out_path = os.path.join(fewshot_dir, "fewshot_results_table.csv")
df.to_csv(out_path, index=False)


   target_task source_type source_tasks  sparsity  seed      mode  train_acc  \
0          qqp    ensemble     sst2+qqp      0.47    42      full   0.640625   
1          qqp    ensemble     sst2+qqp      0.47   123      full   0.640625   
2          qqp    ensemble     sst2+qqp      0.47   999      full   0.640625   
3          qqp    ensemble     sst2+qqp      0.47    42  headonly   0.640625   
4          qqp    ensemble     sst2+qqp      0.47   123  headonly   0.640625   
5          qqp    ensemble     sst2+qqp      0.47   999  headonly   0.578125   
6          qqp     lottery         sst2      0.00    42      full   0.640625   
7          qqp     lottery         sst2      0.00   123      full   0.640625   
8          qqp     lottery         sst2      0.00   999      full   0.640625   
9          qqp     lottery         sst2      0.00    42  headonly   0.640625   
10         qqp     lottery         sst2      0.00   123  headonly   0.640625   
11         qqp     lottery         sst2 

In [17]:
model_dir = "./models"

def flatten_weights(state_dict):
    flat = {}
    for k, v in state_dict.items():
        if not isinstance(v, torch.Tensor):
            continue
        # Ignore biases and embeddings
        if v.ndim >= 2 and "embeddings" not in k and "LayerNorm" not in k:
            flat[k] = v.flatten()
    return flat

def cosine_similarity_models(path_a, path_b):
    wa = torch.load(path_a, map_location="cpu")["model_state_dict"]
    wb = torch.load(path_b, map_location="cpu")["model_state_dict"]

    fa, fb = flatten_weights(wa), flatten_weights(wb)
    sims = []
    for k in fa:
        if k in fb and fa[k].numel() == fb[k].numel():
            sims.append(F.cosine_similarity(fa[k], fb[k], dim=0).item())

    return float(np.mean(sims)) if sims else np.nan

tickets = [f for f in os.listdir(model_dir) if f.startswith("lottery_ticket") and f.endswith(".pt")]
tickets = [f for f in tickets if not "sparsity0" in f]  # exclude dense models

sst2_tickets = [f for f in tickets if "_sst2_" in f]
qqp_tickets  = [f for f in tickets if "_qqp_" in f]

records = []

for sst_file in sst2_tickets:
    m = re.search(r"sparsity(\d+)_seed(\d+)", sst_file)
    if not m:
        continue
    sparsity_sst2, seed = int(m.group(1)) / 100.0, int(m.group(2))

    # Find matching qqp file with same seed
    qqp_file = next((f for f in qqp_tickets if f"seed{seed}" in f), None)
    if qqp_file is None:
        continue

    m2 = re.search(r"sparsity(\d+)_seed(\d+)", qqp_file)
    sparsity_qqp = int(m2.group(1)) / 100.0

    path_sst2 = os.path.join(model_dir, sst_file)
    path_qqp = os.path.join(model_dir, qqp_file)

    sim = cosine_similarity_models(path_sst2, path_qqp)

    records.append({
        "seed": seed,
        "sst2_sparsity": sparsity_sst2,
        "qqp_sparsity": sparsity_qqp,
        "cosine_similarity": sim,
        "sst2_model": sst_file,
        "qqp_model": qqp_file,
    })

df = pd.DataFrame(records)
df = df.sort_values(by="seed").reset_index(drop=True)

print(df)

out_path = os.path.join(model_dir, "cross_task_cosine_similarity.csv")
df.to_csv(out_path, index=False)


   seed  sst2_sparsity  qqp_sparsity  cosine_similarity  \
0    42            0.6           0.7           0.933109   
1   123            0.6           0.7           0.932258   
2   999            0.6           0.7           0.932612   

                                  sst2_model  \
0   lottery_ticket_sst2_sparsity60_seed42.pt   
1  lottery_ticket_sst2_sparsity60_seed123.pt   
2  lottery_ticket_sst2_sparsity60_seed999.pt   

                                  qqp_model  
0   lottery_ticket_qqp_sparsity70_seed42.pt  
1  lottery_ticket_qqp_sparsity70_seed123.pt  
2  lottery_ticket_qqp_sparsity70_seed999.pt  
